# Data Science for Music

## Dataset Sources:

1. My Winamp's music library's Media Library Export - can be found under data/music_library_export.xml
2. My Last.fm account's extracted from [here](https://lastfm.ghan.nl/export/) - can be found under data/lastfm-scrobbles-edchapa.csv
3. [GTZAN Dataset - Music Genre Classification](https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification)

## Process

First, I exported my song library from Winamp, which already contains some information like track name, artist, genre, etc. which I will then use to perform an EDA.

The export is in iTunes XML format, so I will have to convert it into csv format first using Python's xmltodict and csv libraries.

In [23]:
# %pip install xmltodict
%pip install altair


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


First we read the xml file, and create the output csv file.

In [24]:
import xmltodict, csv

with open('data/music_library_export.xml') as xml_file:
    xml_file = xmltodict.parse(xml_file.read())

csv_file = open("data/music_library_export.csv", "w", encoding="utf-8", newline='')
csv_file_writer = csv.writer(csv_file)

We will use the xml file's keys as the column names for the csv file.

In [25]:
xml_file_keys = ["Track ID", "Name", "Artist", "Album Artist", "Album",
                 "Genre", "Comments", "Kind", "Size", "Total Time",
                 "Track Number", "Year", "Bit Rate", "Track Count",
                 "Composer", "Publisher", "Location", "File Folder Count",
                 "Library Folder Count", "Date Modified", "Date Added"]


Now we write a row in the csv file per song in the xml file

In [26]:
csv_file_writer.writerow(xml_file_keys)
for song in xml_file['plist']['dict']['dict']['dict']:
    song_info = ['' for i in range(len(xml_file_keys))]
    skipped_integers = 0
    skipped_strings = 0
    if "Track ID" in song['key']:
        song_info[0] = song['integer'][0]
    else:
        skipped_integers += 1
    if "Name" in song['key']:
        song_info[1] = song['string'][0]
    else:
        skipped_strings += 1
    if "Artist" in song['key']:
        song_info[2] = song['string'][1 - skipped_strings]
    else:
        skipped_strings += 1
    if "Album Artist" in song['key']:
        song_info[3] = song['string'][2 - skipped_strings]
    else:
        skipped_strings += 1
    if "Album" in song['key']:
        song_info[4] = song['string'][3 - skipped_strings]
    else:
        skipped_strings += 1
    if "Genre" in song['key']:
        song_info[5] = song['string'][4 - skipped_strings]
    else:
        skipped_strings += 1
    if "Comments" in song['key']:
        song_info[6] = song['string'][5 - skipped_strings]
    else:
        skipped_strings += 1
    if "Kind" in song['key']:
        song_info[7] = song['string'][6 - skipped_strings]
    else:
        skipped_strings += 1
    if "Size" in song['key']:
        song_info[8] = song['integer'][1 - skipped_integers]
    else:
        skipped_integers += 1
    if "Total Time" in song['key']:
        song_info[9] = song['integer'][2 - skipped_integers]
    else:
        skipped_integers += 1
    if "Track Number" in song['key']:
        song_info[10] = song['integer'][3 - skipped_integers]
    else:
        skipped_integers += 1
    if "Year" in song['key']:
        song_info[11] = song['integer'][4 - skipped_integers]
    else:
        skipped_integers += 1
    if "Bit Rate" in song['key']:
        song_info[12] = song['integer'][5 - skipped_integers]
    else:
        skipped_integers += 1
    if "Track Count" in song['key']:
        song_info[13] = song['integer'][6 - skipped_integers]
    else:
        skipped_integers += 1
    if "Composer" in song['key']:
        song_info[14] = song['string'][7 - skipped_strings]
    else:
        skipped_strings += 1
    if "Publisher" in song['key']:
        song_info[15] = song['string'][8 - skipped_strings]
    else:
        skipped_strings += 1
    if "Location" in song['key']:
        song_info[16] = song['string'][9 - skipped_strings]
    if "File Folder Count" in song['key']:
        song_info[17] = song['integer'][7 - skipped_integers]
    else:
        skipped_integers += 1
    if "Library Folder Count" in song['key']:
        song_info[18] = song['integer'][8 - skipped_integers]
    if "Date Modified" in song['key']:
        song_info[19] = song['date'][0]
    if "Date Added" in song['key']:
        song_info[20] = song['date'][1]
    csv_file_writer.writerow(song_info)
csv_file.close()

Now we will read the csv file into a Polars DataFrame, and display the first rows.

In [27]:
import polars as pl
songs_df = pl.read_csv('data/music_library_export.csv')
songs_df.head()

Track ID,Name,Artist,Album Artist,Album,Genre,Comments,Kind,Size,Total Time,Track Number,Year,Bit Rate,Track Count,Composer,Publisher,Location,File Folder Count,Library Folder Count,Date Modified,Date Added
i64,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,str,str,str,i64,i64,str,str
0,"""A Story Of Boy…","""Mychael Danna …","""Mychael Danna …","""(500) Days Of …","""Soundtrack""",null,"""MPEG audio fil…",1529041,95000,1,2009,128,16,null,null,"""file://localho…",-1,-1,"""2009-11-06T05:…","""2023-10-23T02:…"
1,"""Us""","""Regina Spektor…","""Regina Spektor…","""(500) Days Of …","""Soundtrack""",null,"""MPEG audio fil…",4640293,289000,2,2009,128,16,null,null,"""file://localho…",-1,-1,"""2009-11-06T05:…","""2023-10-23T02:…"
2,"""There Is A Lig…","""The Smiths""","""The Smiths""","""(500) Days Of …","""Soundtrack""",null,"""MPEG audio fil…",3904714,243000,3,2009,128,16,null,null,"""file://localho…",-1,-1,"""2009-11-06T05:…","""2023-10-23T02:…"
3,"""Bad Kids""","""Black Lips""","""Black Lips""","""(500) Days Of …","""Soundtrack""",null,"""MPEG audio fil…",2056471,128000,4,2009,128,16,null,null,"""file://localho…",-1,-1,"""2009-11-06T05:…","""2023-10-23T02:…"
4,"""Please, Please…","""The Smiths""","""The Smiths""","""(500) Days Of …","""Soundtrack""",null,"""MPEG audio fil…",1805315,112000,5,2009,128,16,null,null,"""file://localho…",-1,-1,"""2009-11-06T05:…","""2023-10-23T02:…"


Now I would like to see how many genres we have. For that, we will select the genres column from the DataFrame and display its contents.

In [28]:
songs_df.select(
    pl.col('Genre')
).unique().sort("Genre").to_numpy()


array([[None],
       ['(255)'],
       ['.'],
       ['AOR Classic Rock'],
       ['Alt Rock'],
       ['Alt. Rock'],
       ['Alternative'],
       ['Alternative & Punk'],
       ['Alternative Rock'],
       ['Alternative, Rock'],
       ['Alternativo'],
       ['Ambient'],
       ['Ambient Alternative'],
       ['Ambient Techance'],
       ['Anime'],
       ['Anti-folk'],
       ['Arena/Power Metal'],
       ['Avantgarde'],
       ['Ballad'],
       ['Banda sonora'],
       ['Black Metal'],
       ['Blues'],
       ['Brit Pop'],
       ['Brit-pop'],
       ['Campfire Rock'],
       ['Chiptune'],
       ['Choral'],
       ['Classic Hard Rock'],
       ['Classic Pop Punk'],
       ['Classic Rock'],
       ['Classical'],
       ['Cosmic Tones for Mental Therapy'],
       ['Country'],
       ['Dance'],
       ['Dance & DJ'],
       ['Dance / Disco'],
       ['Death Metal'],
       ['Desert Rock'],
       ['Down-tempo / Pop / Alternativa'],
       ['Dubstep'],
       ['Duck Remixes'],
  

## Data Imputation

We can see there are some genres that repeated but with slight spelling or language differences, so we will try to rename them so they match the rest. We will also remove values that are not actual genres (like 'unknown' for example) and turn them into null values so we can later remove them if necessary.

In [29]:
genres = songs_df.select(
    pl.col("Genre").map_elements(
        lambda x: "Alternative" if x == "Alternativo" else x)
    .map_elements(
        lambda x: "Electronic" if x == "Electronica" else x)
    .map_elements(
        lambda x: "Electronic Pop" if x in ["Pop Electronica", "Electronica / Pop"] else x)
    .map_elements(
        lambda x: "Indie" if x == "indie" else x)
    .map_elements(
        lambda x: "Indie Rock" if x in ["Rock/Indie", "Indie/Rock", "General Indie Rock"] else x)
    .map_elements(
        lambda x: "Miscellaneous" if x == "misc" else x)
    .map_elements(
        lambda x: "Soundtrack" if x in ["soundtrack", "Banda sonora"] else x)
    .map_elements(
        lambda x: "Thrash Metal" if x == "Thrash Metal" else x)
    .map_elements(
        lambda x: "Alt Rock" if x in ["Alt. Rock", "Alternative Rock", "Rock alternativo",
                                      "Alternative, Rock", "General Alternative Rock"] else x)
    .map_elements(
        lambda x: "Brit Pop" if x == "Brit-pop" else x)
    .map_elements(
        lambda x: "Pop Rock" if x in ["Pop/Rock", "Pop/Rock 2000's"] else x)
    .map_elements(
        lambda x: "Pop" if x == "General Pop" else x)
    .map_elements(
        lambda x: "Folk" if x == "General Folk" else x)
    .map_elements(
        lambda x: "Rock" if x in ["General Rock", "Rock En General", "Rock en general", "Rock @",
                                  "rock"] else x)
    .map_elements(
        lambda x: "Heavy Metal" if x == "Rock Duro Y Heavy" else x)
    .map_elements(
        lambda x: "Hip Hop/Rap" if x == "General Rap/Hip-Hop" else x)
    .map_elements(
        lambda x: "Bitpop" if x == "bitpop" else x)
    .map_elements(
        lambda x: "Chillstep" if x == "chillstep" else x)
    .map_elements(
        lambda x: "Chiptune" if x == "chiptune" else x)
    .map_elements(
        lambda x: None if x in ["genre", "default", ".", "(255)", "Other"] else x)
    .map_elements(
        lambda x: "Unclassifiable" if x == "General Unclassifiable" else x)
    .map_elements(
        lambda x: "Soft Rock / Alternative Folk / Folk / Rock" if x == "soft rock/alternative folk/folk/rock" else x)
    .alias("Genre")
).to_series()

songs_df = songs_df.with_columns(genres.alias("Genre"))
songs_df.select(
    pl.col('Genre')
).unique().sort("Genre").to_numpy()


array([[None],
       ['AOR Classic Rock'],
       ['Alt Rock'],
       ['Alternative'],
       ['Alternative & Punk'],
       ['Ambient'],
       ['Ambient Alternative'],
       ['Ambient Techance'],
       ['Anime'],
       ['Anti-folk'],
       ['Arena/Power Metal'],
       ['Avantgarde'],
       ['Ballad'],
       ['Bitpop'],
       ['Black Metal'],
       ['Blues'],
       ['Brit Pop'],
       ['Campfire Rock'],
       ['Chillstep'],
       ['Chiptune'],
       ['Choral'],
       ['Classic Hard Rock'],
       ['Classic Pop Punk'],
       ['Classic Rock'],
       ['Classical'],
       ['Cosmic Tones for Mental Therapy'],
       ['Country'],
       ['Dance'],
       ['Dance & DJ'],
       ['Dance / Disco'],
       ['Death Metal'],
       ['Desert Rock'],
       ['Down-tempo / Pop / Alternativa'],
       ['Dubstep'],
       ['Duck Remixes'],
       ['Duet'],
       ['EDM: Dubstep'],
       ['EDM: Electro House'],
       ['EPM'],
       ['Easy Listening'],
       ['Electro'],
       [

Now we will remove the rows with null values for the Genre and Year columns so we can work with them later.

In [30]:
songs_df = songs_df.drop_nulls(["Genre"])
songs_df = songs_df.drop_nulls(["Year"])

## EDA

Now let's see how many songs we have for each genre.

In [31]:
songs_by_genre = songs_df.select(
    pl.col('Genre')
).to_series().value_counts()
songs_by_genre

Genre,counts
str,u32
"""Indie/Folk""",1
"""Hardcore Punk""",129
"""Alternative""",169
"""Dubstep""",2
"""Gothic Rock""",59
"""Punk""",144
"""Jazz""",24
"""Psychedelic Ro…",10
"""Indie Rock""",87


Now let's get a chart of the top 10 genres with most songs using Altair, and highlight the one with most songs.

In [32]:
import altair as alt

top_10_genres = songs_by_genre.select(
   pl.col("Genre").filter(pl.col("counts") >= pl.col("counts").top_k(10).min()),
   pl.col("counts").filter(pl.col("counts") >= pl.col("counts").top_k(10).min()).alias("Songs")
)
top_genre = top_10_genres.select(
    pl.col("Genre").filter(pl.col("Songs") == pl.col("Songs").max())
).to_numpy()[0][0]
alt.Chart(top_10_genres, title="Top 10 Genres").mark_bar().encode(
    x='Songs',
    y='Genre',
    color=alt.condition(
        alt.datum.Genre == top_genre,
        alt.value('orange'),
        alt.value('steelblue')
    )
)

alt.Chart(...)

Now let's see how many songs we have per Year on an Altair chart, and highlight the one with most songs.

In [33]:
top_10_years = songs_df.filter(pl.col('Year') > 1000).filter(pl.col('Year') < 2024)
top_10_years = top_10_years.select(
    pl.col('Name').alias('Songs'),
    pl.col('Year')
).group_by('Year').agg(
    pl.count('Songs')
).sort(by='Year')
top_year = top_10_years.select(
    pl.col("Year").filter(pl.col("Songs") == pl.col("Songs").max())
).to_numpy()[0][0]
alt.Chart(top_10_years, title="Songs per Year").mark_bar().encode(
    x="Year:O",
    y="Songs:Q",
    color=alt.condition(
        alt.datum.Year == top_year,
        alt.value('orange'),
        alt.value('steelblue')
    )
)


alt.Chart(...)

Now we will load the last.fm export and display the first rows.

In [34]:
scrobbles_df = pl.read_csv("data/lastfm-scrobbles-edchapa.csv")
scrobbles_df.head()

uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid
i64,str,str,str,str,str,str,str
1691282428,"""06 Aug 2023, 0…","""Alejandro Fern…","""""","""Hecho en Méxic…","""19337281-88cb-…","""Caballero""",""""""
1691282200,"""06 Aug 2023, 0…","""Los Acosta""","""ddcbd7c8-73da-…","""Intimidades""","""""","""Como Una Novel…",""""""
1691281988,"""06 Aug 2023, 0…","""Los Askis""","""7941c16f-c2cb-…","""Pasión Y Cumbi…","""""","""Amor Regresa""","""32cf21ce-274a-…"
1691281754,"""06 Aug 2023, 0…","""Los Ángeles Az…","""dcb5e5c6-5f21-…","""De Buenos Aire…","""a2811b27-95b5-…","""Te Necesito""",""""""
1691281575,"""06 Aug 2023, 0…","""Los Ángeles Az…","""dcb5e5c6-5f21-…","""De Buenos Aire…","""a2811b27-95b5-…","""Entrega De Amo…",""""""


Let's create a new column with the "Artist" and "Track" and display the DataFrame.

In [50]:
scrobbles_df = scrobbles_df.with_columns(
    (pl.col('track') + " - " + pl.col('artist')).alias("Song - Artist")
)
scrobbles_df

uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid,Song - Artist
i64,str,str,str,str,str,str,str,str
1691282428,"""06 Aug 2023, 0…","""Alejandro Fern…","""""","""Hecho en Méxic…","""19337281-88cb-…","""Caballero""","""""","""Caballero - Al…"
1691282200,"""06 Aug 2023, 0…","""Los Acosta""","""ddcbd7c8-73da-…","""Intimidades""","""""","""Como Una Novel…","""""","""Como Una Novel…"
1691281988,"""06 Aug 2023, 0…","""Los Askis""","""7941c16f-c2cb-…","""Pasión Y Cumbi…","""""","""Amor Regresa""","""32cf21ce-274a-…","""Amor Regresa -…"
1691281754,"""06 Aug 2023, 0…","""Los Ángeles Az…","""dcb5e5c6-5f21-…","""De Buenos Aire…","""a2811b27-95b5-…","""Te Necesito""","""""","""Te Necesito - …"
1691281575,"""06 Aug 2023, 0…","""Los Ángeles Az…","""dcb5e5c6-5f21-…","""De Buenos Aire…","""a2811b27-95b5-…","""Entrega De Amo…","""""","""Entrega De Amo…"
1691281380,"""06 Aug 2023, 0…","""Los Ángeles Az…","""dcb5e5c6-5f21-…","""De Plaza en Pl…","""""","""Mi Niña Mujer""","""""","""Mi Niña Mujer …"
1691281153,"""06 Aug 2023, 0…","""Grupo Ensamble…","""""","""Tus Jefes No M…","""""","""Tus Jefes No M…","""""","""Tus Jefes No M…"
1691280971,"""06 Aug 2023, 0…","""Grupo Limite""","""""","""Por Puro Amor""","""""","""Yo Sin Tu Amor…","""""","""Yo Sin Tu Amor…"
1691280626,"""06 Aug 2023, 0…","""Los Auténticos…","""""","""Fiesta Naciona…","""2f85b172-b8b6-…","""Amor (Ft. Mon …","""""","""Amor (Ft. Mon …"


Let's get the number of times each song was played, sort it on descending order, and add a row count column.

In [55]:
played_songs = scrobbles_df.group_by('Song - Artist').agg(
    pl.col('Song - Artist').count().alias('play_num')
).sort(by="play_num", descending=True).with_row_count()

played_songs

row_nr,Song - Artist,play_num
u32,str,u32
0,"""Amor a primera…",117
1,"""Icy Skies - Fi…",112
2,"""Calm Down (wit…",109
3,"""On Eloquence -…",96
4,"""Caves - CLANN""",95
5,"""Last Breath - …",93
6,"""Entrega De Amo…",87
7,"""Le Quattro Sta…",85
8,"""Equinox - Eric…",82


Now let's get the top 10 played songs.

In [58]:
top_10_played_songs = played_songs.select(
   pl.col("Song - Artist").filter(pl.col("row_nr") < 10),
   pl.col("play_num").filter(pl.col("row_nr") < 10)
)
top_10_played_songs

Song - Artist,play_num
str,u32
"""Amor a primera…",117
"""Icy Skies - Fi…",112
"""Calm Down (wit…",109
"""On Eloquence -…",96
"""Caves - CLANN""",95
"""Last Breath - …",93
"""Entrega De Amo…",87
"""Le Quattro Sta…",85
"""Equinox - Eric…",82


Now let's chart them using Altair, highlighting the most played song.

In [59]:
top_song = top_10_played_songs.select(
    pl.col("play_num").filter(pl.col("play_num") == pl.col("play_num").max())
).to_numpy()[0][0]
top_10_played_songs
alt.Chart(top_10_played_songs, title="Top 10 Played Songs").mark_bar().encode(
    x=alt.X('play_num', title="Number of times played"),
    y="Song - Artist",
    color=alt.condition(
        alt.datum.play_num == top_song,
        alt.value('orange'),
        alt.value('steelblue')
    )
)

alt.Chart(...)

## Modeling

Now we will import the GTZAN Dataset, which contains non-copyrighted sample tracks of each main genre. 